In [1]:
!pip install seaborn
%matplotlib notebook

You should consider upgrading via the '/Users/dilution/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
import os

In [2]:
from cyvcf2 import VCF
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
# prospax file collection
df_sacs = pd.read_csv("SACS/SACS_all_partners.finaledit.csv", sep="\t")
# variant list formated
df_sacs_extended = pd.read_csv("SACS/SACS.prospax.extended.annotated.tsv", sep="\t")
# sacs_vcf_clinvar = VCF("ext_data/clinvar/05_2021/SACS.clinvar.annotated.vcf")
df_sacs_clinvar = pd.read_csv("SACS/SACS.extended.annotated.tsv", sep="\t")
# gnomad sacs
df_gnom_ad = pd.read_csv("SACS/SACS.gnomad.tsv", sep="\t")
# uniprot_sacs 
uniprot_var = pd.read_csv("SACS/SACS.uniprot.annotated.tsv", sep="\t")

# Pre-Analysis

Data cleaning

## Change df_sacs columns name

In [3]:
columns = [
    "submitter_id",
    "local_case_id",
    "local_family_id",
    "prospax_case_id",
    "ngs_database_id",
    "main_phenotype",
    "case_status",
    "id",
    "gene",
    "chrom",
    "pos",
    "ref",
    "alt",
    "transcript_id",
    "cdna",
    "prot_change",
    "genotype",
    "compound_het_id_s",
    "paxgene_availability",
    "pbmc_availability",
    "fibroblasts_availability",
    "comments",
    "Variant_based_id",
]

df_sacs.columns = columns

## Simplify mutation type

In [4]:
def regroup_csq(x):
    if x in ["3_prime_UTR_variant", "5_prime_UTR_variant"]:
        return "UTR"
    elif x in ["inframe_insertion", "inframe_deletion"]:
        return "Inframe_Indel"
    elif x in ["intron_variant" ]:
        return "Intron"
    elif x in ["missense_variant", "missense_variant&splice_region_variant"]:
        return "Missense"
    elif x in ['splice_region_variant&intron_variant',
              'splice_region_variant&synonymous_variant', 'splice_region_variant&5_prime_UTR_variant']:
        return "Splice_region"
    elif x in ["splice_acceptor_variant", "splice_donor_variant"]:
        return "Canonical_splice"
    elif x in ["start_lost", "stop_gained", "stop_gained&splice_region_variant", 'stop_gained&inframe_insertion']:
        return "Nonsense"
    elif x in ['protein_altering_variant', "frameshift_variant", "frameshift_variant&stop_lost",
               "stop_gained&frameshift_variant", 'splice_donor_variant&splice_acceptor_variant&frameshift_variant&stop_lost&intron_variant',
               'frameshift_variant&splice_region_variant', 'splice_acceptor_variant&splice_donor_variant&frameshift_variant&stop_lost&intron_variant']:
        return "Frameshift"
    elif x in ["synonymous_variant"]:
        return "Synonymous"
    
    
df_sacs_clinvar["csq_minimal"] = df_sacs_clinvar["csq"].apply(lambda x: regroup_csq(x))
df_sacs_extended["csq_minimal"] = df_sacs_extended["csq"].apply(lambda x: regroup_csq(x))
df_gnom_ad["csq_minimal"] = df_gnom_ad["mutation_type"].apply(lambda x: regroup_csq(x))

## ADD variant_based_id column to files

In [5]:
df_sacs_clinvar["Variant_based_id"] = df_sacs_clinvar[["chrom", "pos", "ref", "alt"]].apply(lambda x: "-".join(x.values.astype(str)), axis=1)
df_sacs_extended["Variant_based_id"] = df_sacs_extended[["chrom", "pos", "ref", "alt"]].apply(lambda x: "-".join(x.values.astype(str)), axis=1)
df_gnom_ad.rename(columns={"variant_id":"Variant_based_id"}, inplace=True)


## Simplify case status 
unsure and not solved could be re-grouped

In [6]:
# get variant set(ids) from different case status
solved_ids = set(df_sacs[df_sacs['case_status']=="solved SACS"]["Variant_based_id"])
unsure_notsolved_ids = set(df_sacs[df_sacs['case_status'].isin(["not solved", "unsure"])]["Variant_based_id"])
solved_other_ids = set(df_sacs[df_sacs['case_status']=="solved other gene"]["Variant_based_id"])

## ADD protein domain info

In [7]:
import hgvs.parser
import numpy as np

def clean_hgvsp(x):
    if x=="n/a" or x=="-1":
        return "n/a"
    else:
        return x.replace("%3D", "=")
        
def short_prot_desc(x):
    if x=="n/a":
        return "n/a"
    else:
        return x.split(":")[1]

def get_prot_pos(x, hp):
    if x=="n/a":
        return -1
    else:
        p = hp.parse_hgvs_variant(x)
        return p.posedit.pos.start.base
    

    
hp = hgvs.parser.Parser()

# list sacs prospax variant
df_sacs_extended["hgvsp"].fillna("n/a", inplace=True)
df_sacs_extended["hgvsp_clean"] = df_sacs_extended["hgvsp"].apply(lambda x: clean_hgvsp(x))
df_sacs_extended["hgvsp_short"] = df_sacs_extended["hgvsp_clean"].apply(lambda x: short_prot_desc(x))
df_sacs_extended["protein_pos"] = df_sacs_extended["hgvsp_clean"].apply(lambda x: get_prot_pos(x, hp))

# list clinvar variant
df_sacs_clinvar["hgvsp"].fillna("n/a", inplace=True)
df_sacs_clinvar["hgvsp_clean"] = df_sacs_clinvar["hgvsp"].apply(lambda x: clean_hgvsp(x))
df_sacs_clinvar["hgvsp_short"] = df_sacs_clinvar["hgvsp_clean"].apply(lambda x: short_prot_desc(x))
df_sacs_clinvar["protein_pos"] = df_sacs_clinvar["hgvsp_clean"].apply(lambda x: get_prot_pos(x, hp))

# list gnomad variant
df_gnom_ad["hgvsp"].fillna("n/a", inplace=True)
df_gnom_ad["hgvsp_clean"] = df_gnom_ad["hgvsp"].apply(lambda x: clean_hgvsp(x))
df_gnom_ad["hgvsp_short"] = df_gnom_ad["hgvsp_clean"].apply(lambda x: short_prot_desc(x))
df_gnom_ad["protein_pos"] = df_gnom_ad["hgvsp_clean"].apply(lambda x: get_prot_pos(x, hp))


# list gnomad variant
uniprot_var["hgvsp"].fillna("n/a", inplace=True)
uniprot_var["hgvsp_clean"] = uniprot_var["hgvsp"].apply(lambda x: clean_hgvsp(x))
uniprot_var["hgvsp_short"] = uniprot_var["hgvsp_clean"].apply(lambda x: short_prot_desc(x))
uniprot_var["protein_pos"] = uniprot_var["hgvsp_clean"].apply(lambda x: get_prot_pos(x, hp))


In [8]:
sacs_domain = pd.read_csv("SACS/sacs.bed", sep=" ", header=None)

In [9]:
import bisect

# map of regions
sacs_prot = {
    "0_inter": (1, 8),
    "1_ubq_like": (9, 83),
    "2_sr1_SIRPT1": (84, 339),
    "3_SRR1": (340, 399),
    "4_sr2_SIRPT1": (400, 557),
    "5_inter": (558, 643),
    "6_srX_SIRPT1": (644, 1162),
    "7_inter": (1163, 1220),
    "8_sr3_SIRPT1": (1221, 1374),
    "9_interinter": (1375, 1443),
    "10_sr1_SIRPT2": (1444, 1747),
    "11_SRR2": (1748, 1825),
    "12_sr2_SIRPT2": (1826, 1968),
    "13_inter": (1969, 2286),
    "14_sr3_SIRPT3": (2287, 2443),
    "15_inter": (2444, 2511),
    "16_sr1_SIRPT3": (2512, 2768),
    "17_SRR3": (2769, 2825),
    "18_sr2_SIRPT3": (2826, 2960),
    "19_inter": (2961, 3080),
    "20_srX_SIRPT3": (3081, 3659),
    "21_inter": (3660, 3735),
    "22_sr3_SIRPT3": (3736, 3896),
    "23_inter": (3897, 4305),
    "24_dnaj": (4306, 4393),
    "25_inter": (4394, 4450),
    "26_hepn": (4451, 4567),
    "27_inter": (4568, 4600)
}

# reduce to categories
ref = {
    "0_inter": (0, [0, 5, 7, 9, 13, 15, 19, 21, 23, 25, 27]),
    "1_ubq_like": (1, [1]),
    "2_sr1_histidine_kinase": (2, [2, 10, 16]),
    "3_sr2": (3, [4, 12, 18]),
    "4_SRR": (4, [3, 11, 17]),
    "5_srx": (5, [6, 20]),
    "6_sr3": (6, [8, 14, 22]),
    "7_dnaj": (7, [24]),
    "8_hepn": (8, [26])
}

def get_domain_name(x, map_dom):
    
    if x==-1:
        return -1
    else:
        intervals = sorted([v[1] for i, v in map_dom.items()])
        intervals_name = [i for i in map_dom]
        domain = bisect.bisect_left(intervals, x)
        domain_name = intervals_name[domain]
        return domain_name

def get_domain_cat(x, map_dom, ref):
    if x==-1:
        return -1
    else:
        intervals = sorted([v[1] for i, v in map_dom.items()])
        # intervals_name = [i for i in map_dom]
        domain = bisect.bisect_left(intervals, x)
        for i, v in ref.items():
            if domain in v[1]:
                return v[0]


In [10]:
# apply to df_var
df_sacs_extended["domain_name"] = df_sacs_extended["protein_pos"].apply(lambda x: get_domain_name(x, sacs_prot))
df_sacs_extended["cat_domain"] = df_sacs_extended["protein_pos"].apply(lambda x: get_domain_cat(x, sacs_prot, ref))

# clinvar
df_sacs_clinvar["domain_name"] = df_sacs_clinvar["protein_pos"].apply(lambda x: get_domain_name(x, sacs_prot))
df_sacs_clinvar["cat_domain"] = df_sacs_clinvar["protein_pos"].apply(lambda x: get_domain_cat(x, sacs_prot, ref))


# gnomad
df_gnom_ad["domain_name"] = df_gnom_ad["protein_pos"].apply(lambda x: get_domain_name(x, sacs_prot))
df_gnom_ad["cat_domain"] = df_gnom_ad["protein_pos"].apply(lambda x: get_domain_cat(x, sacs_prot, ref))

# uniprot
uniprot_var["domain_name"] = uniprot_var["protein_pos"].apply(lambda x: get_domain_name(x, sacs_prot))
uniprot_var["cat_domain"] = uniprot_var["protein_pos"].apply(lambda x: get_domain_cat(x, sacs_prot, ref))


# Preparatory steps




## 1 - Annotate variants uniformely --> (done with VEP)

## 2 - Identify and flag likely pathogenic variants
- All rare truncating variants (nonsense, frameshift, canonical splice)
- All ClinVar likely pathogenic / pathogenic variants


In [11]:
def flag_lp_p(x, list_v_id):
    if x in list_v_id:
        return "y"
    else:
        return "n"

In [12]:
# get truncating mutation 
truncating_type = ['Nonsense', 'Frameshift', 'Canonical_splice']
list_v_id_truncating = df_sacs_extended[df_sacs_extended["csq_minimal"].isin(truncating_type)]["Variant_based_id"].unique()

In [13]:
# get likely pathogenic and pathogenic from clinvar and uniprot
# inner merge clinvar prospax
# assertion considered are: 
# 'reviewed_by_expert_panel',
# 'criteria_provided,_single_submitter',
# 'criteria_provided,_multiple_submitters,_no_conflicts'
inner_merge_clinvar_prospax = pd.merge(df_sacs_clinvar[["Variant_based_id", 'clinsig', 'clinsigconflict','clinsigincl']], df_sacs_extended, how="inner", on=["Variant_based_id"])
lp_p_labels = ['Likely_pathogenic', 'Pathogenic/Likely_pathogenic', 'Pathogenic']
lp_p_clinvar = inner_merge_clinvar_prospax[inner_merge_clinvar_prospax["clinsig"].isin(lp_p_labels)]["Variant_based_id"].unique()

# inner merge uniprot prospax 
# pathogenic are labeled as 2 in clinsig col
inner_merge_uniprot_prospax = pd.merge(uniprot_var[["Variant_based_id", 'clinsig']], df_sacs_extended, how="inner", on=["Variant_based_id"])
lp_p_uniprot = inner_merge_uniprot_prospax[inner_merge_uniprot_prospax["clinsig"]==2]["Variant_based_id"].unique()


In [14]:
# add gross indels to list of lp_p variants
gross_indels = list(df_sacs[df_sacs["ref"].isin(["dup", "del"])]["Variant_based_id"].unique())

In [15]:
# merge lists lp/p uniprot / clinvar / truncating variants
# and flag them in variant collection
final_lp_p_id = list(set(list(lp_p_clinvar) + list(lp_p_uniprot) + list(list_v_id_truncating) + gross_indels))
df_sacs_extended["is_lp_p"] = df_sacs_extended["Variant_based_id"].apply(lambda x: flag_lp_p(x, final_lp_p_id))

In [16]:
# a
print("Number of variants being reported as LP/P in clinvar/uniprot: {}".format(df_sacs_extended[df_sacs_extended["is_lp_p"]=="y"].shape[0]))

Number of variants being reported as LP/P in clinvar/uniprot: 125


In [17]:
print("Number of variants being reported as LP/P in clinvar/uniprot + gross deletions: {}".format(len(final_lp_p_id)))

Number of variants being reported as LP/P in clinvar/uniprot + gross deletions: 129


## 3 - Filter remaining variants uniformely
- Keep all likely pathogenic variants
- Filter criteria for remaining variants: 
    - MAF < 0.5%
    - highly conserved (missense only: PhasCons ≥ 0.7, GERP ≥ 2) (to be discussed)

In [18]:
def filter_maf(df, threshold=0.005):
    df_filt = df[(df['thKg']<=threshold) &
                 (df['gnomad_ex']<=threshold) &
                 (df['gnomad_gn']<=threshold)
                ]
    return df_filt

def filter_conservation(df, phastcons_min=0.7, gerp_min=2):
    df_filt = df[((df['gerp_rs']>=gerp_min) | (df['gerp_rs'].isnull())) &
                 ((df['phastcons']>=phastcons_min) | (df['phastcons'].isnull())) 
                ]
    return df_filt

In [19]:
df_not_lp_p = df_sacs_extended[df_sacs_extended["is_lp_p"]=="n"]
df_not_lp_p.shape

(269, 55)

In [20]:
# extract the non-LP/P variant from preparatory steps
# filter on MAF<=0.5%
df_freq1 = filter_maf(df_not_lp_p)
df_freq_conserver_filtered = filter_conservation(df_freq1)
final_conserved_variant_id = list(df_freq_conserver_filtered["Variant_based_id"].unique())

In [21]:
# flag variants if lp/p - conserved - none
def flag_conserved(x, list_lp_p, list_rare_conserved):
    if x in list_rare_conserved:
        return "rare_conserved"
    elif x in list_lp_p:
        return "lp_p"
    else:
        return "not_conserved"
    
    
df_sacs_extended["is_conserved"] = df_sacs_extended["Variant_based_id"].apply(lambda x: flag_conserved(x, final_lp_p_id, final_conserved_variant_id))

In [22]:
df_freq_conserver_filtered.shape

(197, 55)

# A - Tier 1: Cases carrying 2 likely pathogenic variants

## 4 - Identify cases carrying two (likely) pathogenic variants
- flag cases as “solved”
- filter out all variants in solved cases
- collect phenotypic information, perform segregation analysis, consider functional assays (e.g. as positive controls)


In [23]:
# create unique patient (submitter-localsubjectid)
df_lp_p = df_sacs[df_sacs["Variant_based_id"].isin(final_lp_p_id)]

unique_case_list = set([(row['submitter_id'],row['local_case_id']) 
                    for idx, row in df_lp_p.iterrows()])

In [24]:
def stream_cases(cases, df):
    for case in cases:
        yield((case, df[(df['local_case_id']==case[1]) & \
                 (df['submitter_id']==case[0])]))

# check if patient has one gross dup/del        
def is_large_dup_del(df):
    if "dup" in df['ref'].values:
        return True
    elif "del" in df['ref'].values:
        return True
    else:
        False
        
# check if patient has 2 lp/p variants       
def is_2_lp_p(df, list_lp_p):
    if df[df["Variant_based_id"].isin(list_lp_p)].shape[0]>=2:
        return True
    else:
        return False

# check if patient has 1 lp/p variant + one gross dup/del
def is_lpp_del_dup(df, list_lp_p):
    if df[df["Variant_based_id"].isin(list_lp_p)].shape[0]>1:
        return True
    else:
        return False
    
# check if patient has Hom lp/p variant
def is_lp_p_hom(df, list_lp_p):
    if df[(df["Variant_based_id"].isin(list_lp_p)) &
       (df["genotype"]=="Hom")].empty:
        return False
    else:
        return True
    
def tier1_screen_case(df, list_lp_p):
    # if one variant from lp/p list only variant hom
    if df.shape[0]==1:
        return True
    # if more than one variant in case
    elif df.shape[0]>1:
        # if on variant is a gross del/dup
        if is_large_dup_del(df):
            # if one gross dup/del + one lp/p variant
            if is_lpp_del_dup(df, list_lp_p):
                return True
            else:
                # if just one gross dup/del
                return False
        # if 2 lp/p het / caveats ID27288 + 357-987-169
        elif is_2_lp_p(df, list_lp_p):
            return True
        # if one HOM lp/p variants and other non het/HOM non-lp/p variants
        elif is_lp_p_hom(df, list_lp_p):
            return True
        else:
            return False
            
            
        
        

### Test

In [25]:
test_dup_del_lp = df_sacs[df_sacs["local_case_id"]=="ARSACS11"]    
test_2_lp_p = df_sacs[df_sacs["local_case_id"]=="ARSACS9"]  
test_not_2_lp = df_sacs[df_sacs["local_case_id"]=="P3_ID7"]
test_dup_del_no_lp = df_sacs[df_sacs["local_case_id"]=="156-038-303"]
test_random = df_sacs[df_sacs["local_case_id"]=="FN14062"]
test_hom_many = df_sacs[df_sacs["local_case_id"]=="SCA1511"]

In [26]:
# true
tier1_screen_case(test_dup_del_lp, final_lp_p_id)

True

In [27]:
# true
tier1_screen_case(test_2_lp_p, final_lp_p_id)

True

In [28]:
# false
tier1_screen_case(test_not_2_lp, final_lp_p_id)

False

In [29]:
# false
tier1_screen_case(test_dup_del_no_lp, final_lp_p_id)

False

In [30]:
# false
tier1_screen_case(test_random, final_lp_p_id)

False

In [31]:
# true
tier1_screen_case(test_hom_many, final_lp_p_id)

True

### Run on all cases

In [25]:
# collect case local id having 2 lp/p variants or 1 lp/p variant + gross del/dup
solved_cases = []
for case in stream_cases(unique_case_list, df_sacs):
    patient = case[0]
    df = case[1]
    if tier1_screen_case(df, final_lp_p_id):
        solved_cases.append(patient)
    else:
        pass

In [26]:
# extract cases from prospax collection
# flag them in df_sacs
def is_solved(x, solved_id):
    if x in solved_id:
        return "y"
    else:
        return "n"
solved_local_ids = {case[1] for case in solved_cases}
df_sacs["tier1_is_solved"] = df_sacs["local_case_id"].apply(lambda x: is_solved(x, solved_local_ids))
df_solved = df_sacs[df_sacs["local_case_id"].isin(solved_local_ids)]

In [34]:
print("Number of unique cases being tier1: {}".format(df_solved["local_case_id"].unique().shape[0]))

Number of unique cases being tier1: 152


In [44]:
# save cases being tier 1
df_solved.to_csv("SACS/tier1/tier1_solved_case.tsv", sep="\t", index=False)
df_solved.to_excel("SACS/tier1/tier1_solved_case.xlsx", index=False, freeze_panes=(1,4))

In [27]:
# extract variant list being lp/p and Tier1
tiers1_variant_id = df_solved[df_solved["Variant_based_id"].isin(final_lp_p_id)]["Variant_based_id"].unique()
tiers1_variants = df_sacs_extended[df_sacs_extended["Variant_based_id"].isin(tiers1_variant_id)]

In [45]:
# saving tier1: lp-p variants 
tiers1_variants.to_csv("SACS/tier1/tiers_solved_variants.tsv", sep="\t", index=False)
tiers1_variants.to_excel("SACS/tier1/tiers_solved_variants.xlsx", index=False, freeze_panes=(1,4))

In [28]:
# number of solved cases
len(df_solved["local_case_id"].unique())

152

In [29]:
len(df_solved["Variant_based_id"].unique())

126

In [58]:
# any benign among Tier1??
[v for v in tiers1_variants["Variant_based_id"].unique() if v in final_list_benign]

[]

# B - Tier 2: Cases carrying 1 likely pathogenic variants in combination with a VUS


## 5 -  Identify cases carrying 1 likely pathogenic variant in combination with at least 1 additional variant
- depending on the number of remaining cases, proceed to functional follow up or consider additional filtering steps
- criteria supporting pathogenicity: 
    - phenotype matches SACS/SPG7
    - variants segregate in the family
    - highly conserved, predicted deleterious (to be discussed)
    - missense variant is located in a functional domain
    - variant has functional effect (splicing, downregulation of protein, downstream functional effect)


In [30]:
# create unique patient (submitter-localsubjectid)
df_tier2_prep = df_sacs[df_sacs["tier1_is_solved"]=="n"]
print(df_tier2_prep.shape)
df_tier2_prep = df_tier2_prep[df_tier2_prep["Variant_based_id"].isin(final_lp_p_id)]
print(df_tier2_prep.shape)
tier2_unique_case_list = set([(row['submitter_id'],row['local_case_id']) 
                    for idx, row in df_tier2_prep.iterrows()])


(522, 24)
(34, 24)


** remember to use non-lp/p conserved&rare variants

In [31]:
rare_conserved_variants = df_freq_conserver_filtered["Variant_based_id"].unique()

** create list of benign-likelybenign variants to investigate

In [32]:
benign_clinvar = list(df_sacs_clinvar[df_sacs_clinvar["clinsig"].isin(['Likely_benign', 'Benign', 'Benign/Likely_benign'])]["Variant_based_id"].unique())

# remove 2 uniprot variants which are VUS
benign_uniprot = list(uniprot_var[uniprot_var["clinsig"]==1]["Variant_based_id"].unique())
benign_uniprot.remove("13-23906766-T-C")
benign_uniprot.remove("13-23912630-C-T")

In [33]:
final_list_benign = list(set(benign_clinvar + benign_uniprot))
# add flag in main variant collection - is_variant_benign reported
def is_benign_uniprot_clinvar(x, list_benign):
    if x in list_benign:
        return "y"
    else:
        return "n"
    
df_sacs_extended["has_benign_report"] = df_sacs_extended["Variant_based_id"].apply(lambda x: is_benign_uniprot_clinvar(x, final_list_benign))

In [34]:
def stream_cases(cases, df):
    for case in cases:
        yield((case, df[(df['local_case_id']==case[1]) & \
                 (df['submitter_id']==case[0])]))
        
def get_lp_p_variant(df, list_lp_p):
    list_df_lp_p = list(df[df["Variant_based_id"].isin(list_lp_p)]["Variant_based_id"])
    return list_df_lp_p

def get_rare_conserved_variant(df, list_conserved_rare):
    list_df_conserverd_rare = list(df[df["Variant_based_id"].isin(list_conserved_rare)]["Variant_based_id"])
    return list_df_conserverd_rare

def get_benign_variant(df, list_benign):
    list_df_benign = list(df[df["Variant_based_id"].isin(list_benign)]["Variant_based_id"])
    return list_df_benign

def get_all_var_not_lpp(df, v_lp_p):
    v_not_lpp = [v for v in df["Variant_based_id"].unique() if v not in v_lp_p]
    return v_not_lpp

def get_hyp_vus(v_not_lpp, v_benign, v_rare_conserved):
    hyp_vus = [v for v in v_not_lpp if v not in v_benign and v in v_rare_conserved]
    return hyp_vus

def is_tier2(df, df_length, v_lp_p, v_benign, v_rare_conserved):
    if df_length==2:
        if len(v_lp_p)==1 and len(v_benign)==0 and len(v_rare_conserved)==1:
            return True
        else:
            return False
    else:
        not_lpp_v = get_all_var_not_lpp(df, v_lp_p)
        hyp_vus = get_hyp_vus(not_lpp_v, v_benign, v_rare_conserved)
        if hyp_vus:
            return True
        else: 
            return False
        
def tier2_screen_case(df, list_lp_p, list_conserved_rare, list_benign):
    if df.shape[0]==1:
        print("problem 1 variant only", list(df["Variant_based_id"]))
    else:
        v_lp_p = get_lp_p_variant(df, list_lp_p)
        # print("lp_p", len(v_lp_p))
        v_benign = get_benign_variant(df, list_benign)
        # print("benign", len(v_benign))
        v_rare_conserved = get_rare_conserved_variant(df, list_conserved_rare)
        # print("conserved_rare", len(v_rare_conserved))
        df_length = df.shape[0]
        
        return is_tier2(df, df_length, v_lp_p, v_benign, v_rare_conserved)
            
        
        
            
        

In [35]:
tier2_cases = []
tier2_cases_rejected = []
for case in stream_cases(tier2_unique_case_list, df_sacs):
    patient = case[0]
    df = case[1]
    #if patient[1]=="ATX81":
        #print(df.shape)
        
        #print(tier2_screen_case(df, final_lp_p_id, final_conserved_variant_id, final_list_benign))
    if tier2_screen_case(df, final_lp_p_id, final_conserved_variant_id, final_list_benign):
        tier2_cases.append(patient)
    else:
        tier2_cases_rejected.append(patient)
        print(patient, "not tier2")

('Pisa', '279-716-956') not tier2
('P3 - Nijmegen', 'P3_ID28') not tier2
('P6 - Istanbul', 'MYO47') not tier2


In [36]:
# extract cases from prospax collection
# flag them in df_sacs
def flag_tier2(x, tier2_id, tier2_cases_rejected):
    if x in tier2_id:
        return "y"
    elif x in tier2_cases_rejected:
        return "rejected_automatic_not_rare/conserved_or_benign_report"
    else:
        return "n"

def flag_tier2_variant_lvl(x, lp_p, rare_conserved, benign):
    if x in lp_p:
        return "lp_p_variant"
    elif x in rare_conserved and x not in benign:
        return "rare_conserved_variant"
    elif x not in rare_conserved and x not in benign:
        return "NOT_rare_conserved_variant"
    elif x not in rare_conserved and x in benign:
        return "NOT_rare_conserved_variant_AND_benign_report"
    
tier2_local_ids = {case[1] for case in tier2_cases}
tier2_rejected_local_ids = {case[1] for case in tier2_cases_rejected}
df_sacs["is_tier2"] = df_sacs["local_case_id"].apply(lambda x: flag_tier2(x, tier2_local_ids, tier2_rejected_local_ids))
df_tier2 = df_sacs[df_sacs["local_case_id"].isin(tier2_local_ids)]
df_tier2["variant_level_status"] = df_tier2["Variant_based_id"].apply(lambda x: flag_tier2_variant_lvl(x, final_lp_p_id, final_conserved_variant_id, final_list_benign))
df_tier2_rejected = df_sacs[df_sacs["local_case_id"].isin(tier2_rejected_local_ids)]
df_tier2_rejected["variant_level_status"] = df_tier2_rejected["Variant_based_id"].apply(lambda x: flag_tier2_variant_lvl(x, final_lp_p_id, final_conserved_variant_id, final_list_benign))

/Users/dilution/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/dilution/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [37]:
# create column to flag questionable variants 
def flag_questionable(x):
    if x == "lp_p_variant":
        return 0
    elif x == "rare_conserved_variant":
        return 2
    elif x in ["NOT_rare_conserved_variant", "NOT_rare_conserved_variant_AND_benign_report"]:
        return 1
    
df_tier2["is_questionable"] = df_tier2["variant_level_status"].apply(lambda x: flag_questionable(x))
df_tier2_rejected["is_questionable"] = df_tier2_rejected["variant_level_status"].apply(lambda x: flag_questionable(x))

/Users/dilution/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/dilution/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [38]:
df_tier2_variants = df_sacs_extended[df_sacs_extended["Variant_based_id"].isin(df_tier2["Variant_based_id"].unique())]

In [39]:
def flag_questionable_var(x):
    if x == "lp_p":
        return 0
    else:
        return 1
df_tier2_variants["is_questionable_variant"] = df_tier2_variants["is_conserved"].apply(lambda x: flag_questionable_var(x))

/Users/dilution/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
df_tier2_variants.to_csv("SACS/tier2/tiers2_cases_variants.tsv", sep="\t", index=False)
df_tier2_variants.to_excel("SACS/tier2/tiers2_cases_variants.xlsx", index=False, freeze_panes=(1,4))
df_tier2.to_csv("SACS/tier2/tier2_cases.tsv", sep="\t", index=False)
df_tier2.to_excel("SACS/tier2/tier2_cases.xlsx", index=False, freeze_panes=(1,4))
df_tier2_rejected.to_csv("SACS/tier2/tier2_rejected_cases.tsv", sep="\t", index=False)
df_tier2_rejected.to_excel("SACS/tier2/tier2_rejected_cases.xlsx", index=False, freeze_panes=(1,4))

In [40]:
len(df_tier2["local_case_id"].unique())

31

### Tests

In [68]:
# False
test_one_lp_benign = df_sacs[df_sacs["local_case_id"]=="MYO47"]
tier2_screen_case(test_one_lp_benign, final_lp_p_id, final_conserved_variant_id, final_list_benign)

False

In [69]:
# True
test_true = df_sacs[df_sacs["local_case_id"]=="AAR-BOR-CLA-334"]
tier2_screen_case(test_true, final_lp_p_id, final_conserved_variant_id, final_list_benign)

True

In [70]:
# True 
test_complex = df_sacs[df_sacs["local_case_id"]=="ATX81"]
tier2_screen_case(test_complex, final_lp_p_id, final_conserved_variant_id, final_list_benign)

True

# C - Tier 3: Cases carrying 2 VUS

## 6 - Identify cases carrying 2 variants of unknown significance
- apply standard filters proposed by Robin (see next slide)
- depending on the number of remaining cases, proceed to functional follow up or consider additional filtering steps
- criteria supporting pathogenicity: 
    - phenotype matches SACS/SPG7
    - variants segregate in the family
    - missense variant is located in a functional domain
    - variant has functional effect (splicing, downregulation of protein, downstream functional effect)


In [46]:
# get cases which are not included in tier1 and tier2
df_tier3_prep = df_sacs[(df_sacs["tier1_is_solved"]=="n") & (df_sacs["is_tier2"]=="n")]

print("Number of cases to investigate in the context of Tier3 analysis: {}".format(len(df_tier3_prep["local_case_id"].unique())))

Number of cases to investigate in the context of Tier3 analysis: 177


In [47]:
# cases to investigate
tier3_unique_case_list = df_tier3_prep["local_case_id"].unique()
# get variants id tier3
variants_tier3_unique_cases = df_tier3_prep["Variant_based_id"].unique()

In [48]:
# get variants from tier3_unique_case_list
df_pre_tier3_variants = df_sacs_extended[df_sacs_extended["Variant_based_id"].isin(variants_tier3_unique_cases)]

print("Number of variants to investigate in the context of Tier3 analysis: {}".format(df_pre_tier3_variants.shape[0]))

Number of variants to investigate in the context of Tier3 analysis: 238


In [49]:
# separate variants according to mutation type
# exonic / no synonymous / no splice variants region / checked they are all intronic atm
exonic = df_pre_tier3_variants[df_pre_tier3_variants["csq_minimal"].isin(["Missense", "Inframe_Indel"])]

# missense
missense_pre_tier3 = df_pre_tier3_variants[df_pre_tier3_variants["csq_minimal"]=="Missense"]

# inframe indel
inframe_indels_tier3 = df_pre_tier3_variants[df_pre_tier3_variants["csq_minimal"]=="Inframe_Indel"]

# untranslated_region
untranslated_regions_tier3 = df_pre_tier3_variants[df_pre_tier3_variants["csq_minimal"].isin(['UTR', 'Intron', 'Splice_region'])]

# synonymous
synonymous = df_pre_tier3_variants[df_pre_tier3_variants["csq_minimal"]=="Synonymous"]


In [50]:
len(exonic["Variant_based_id"].unique())

105

### apply filtration to whole pre_tier3 collection (considering all mutation type)
- two filtration:
    - deleteriouness 
    - spliceai

In [51]:
# create filtration 
def filter_tier3(df):
    df = df[((df['metalr_pred'] == "D") | (df['metalr_pred'].isnull())) &
           ((df['m_cap_pred'] == "D") | (df['m_cap_pred'].isnull())) &
           ((df['sift']=="deleterious") | (df['sift'].isnull())) &
           ((df['polyphen'].isin(["probably_damaging", "possibly_damaging"]))  | (df['polyphen'].isnull())) &
           ((df['mutationtaster_pred']=='D&D&N') | (df['mutationtaster_pred']=='D&D&D') | (df['mutationtaster_pred']=='A&A&D') | (df['mutationtaster_pred'].isnull())) &
           ((df['cadd']>=(15)) | (df['cadd'].isnull())) & 
           ((df['dann']>=(0.98)) | (df['dann'].isnull())) &
           ((df['revel']>=(0.5)) | (df['revel'].isnull())) &
           ((df['provean_pred'].isin(["D&D", "D&N", "D", "N&D"])) | (df['provean_pred'].isnull()))]
    return df

def filter_spliceai(df, min_score=0.2):
    df = df[
            (df['spliceai_pred_DS_AG']>=min_score) |
            (df['spliceai_pred_DS_AL']>=min_score) |
            (df['spliceai_pred_DS_DG']>=min_score) |
            (df['spliceai_pred_DS_DL']>=min_score) 
    ]
    return df

# main filtration
df_filtered_pre_tier3 = filter_tier3(exonic)
    
# spliceai filtration
df_splice_filtered_pre_tier3 = filter_spliceai(df_pre_tier3_variants)

In [52]:
len(df_splice_filtered_pre_tier3["Variant_based_id"].unique())

1

In [53]:
# merge splice and filtered variants filtered
df_filtered_tier3_stage1 = pd.concat([df_filtered_pre_tier3, df_splice_filtered_pre_tier3])

In [54]:
# from filtered list fetch cases with variants
filtered_variants_tiers3 = df_filtered_tier3_stage1["Variant_based_id"].unique()
# get cases id
cases_filtered_tier3_stage1 = df_tier3_prep[df_tier3_prep["Variant_based_id"].isin(filtered_variants_tiers3)]["local_case_id"].unique()
# get df
df_tier3_cases_stage1 = df_tier3_prep[df_tier3_prep["local_case_id"].isin(cases_filtered_tier3_stage1)]

# did variant pass filtration?
def flag_variant_filter(x, pass_v_l):
    if x in pass_v_l:
        return "y"
    else: 
        return "n"
    
df_tier3_cases_stage1["pass_tier3_filtration"] = df_tier3_cases_stage1["Variant_based_id"].apply(lambda x: flag_variant_filter(x, filtered_variants_tiers3))

/Users/dilution/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [55]:
# save variants passing filtration
df_filtered_tier3_stage1.to_csv("SACS/tier3/stage1_tier3_variants.tsv", sep="\t", index=False)
df_filtered_tier3_stage1.to_excel("SACS/tier3/stage1_tier3_variants.xlsx", index=False, freeze_panes=(1,4))
# save cases carrying variant
df_tier3_cases_stage1.to_csv("SACS/tier3/stage1_tier3_cases.tsv", sep="\t", index=False)
df_tier3_cases_stage1.to_excel("SACS/tier3/stage1_tier3_cases.xlsx", index=False, freeze_panes=(1,4))

In [56]:
len(df_tier3_cases_stage1["local_case_id"].unique())

39

In [57]:
len(df_filtered_tier3_stage1["Variant_based_id"].unique())

30

In [102]:
# 
df_sacs[df_sacs["case_status"]=="solved SACS"].drop_duplicates(subset=["submitter_id", "local_case_id"])

,submitter_id,local_case_id,local_family_id,prospax_case_id,ngs_database_id,main_phenotype,case_status,id,gene,chrom,...,prot_change,genotype,compound_het_id_s,paxgene_availability,pbmc_availability,fibroblasts_availability,comments,Variant_based_id,tier1_is_solved,is_tier2
0,P1 - Tübingen,ID16341,FN16341,064-258-853,TreatHSP,spastic ataxia,solved SACS,P1_SACS_Tubingen_1,SACS,13,...,p.Gly1275Ter,Het,P1_SACS_Tubingen_2,no,no,no,NaN,13-23914246-C-A,y,n
2,P1 - Tübingen,ID23075,FN23075,167-759-767,TreatHSP,spastic ataxia,solved SACS,P1_SACS_Tubingen_3,SACS,13,...,p.Arg1575Trp,Hom,NaN,yes,no,no,NaN,13-23913292-G-A,n,n
3,P1 - Tübingen,ID24107,FN23075,444-749-699,TreatHSP,spastic ataxia,solved SACS,P1_SACS_Tubingen_4,SACS,13,...,p.Arg1575Trp,Hom,NaN,no,no,no,NaN,13-23913292-G-A,n,n
4,P1 - Tübingen,ID27288,FN27286,493-515-735,TreatHSP,spastic ataxia,solved SACS,P1_SACS_Tubingen_5,SACS,13,...,p.Thr2388Argfs*10,Hom,NaN,no,no,no,NaN,13-23910851-GGT-G,y,n
27,P1 - Tübingen,ATX553,ATX553,NaN,Genesis,ataxia,solved SACS,P1_SACS_Tubingen_28,SACS,13,...,p.Arg3903Ter,Hom,NaN,unknown,unknown,unknown,NaN,13-23906308-G-A,y,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,P7 - Paris,AAR-STR-CHA-314-1,AAR-STR-CHA-314,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_84,SACS,13,...,p.Asp202Gly,Het,P7_SACS_Paris_85,no,yes,no,NaN,13-23930146-T-C,n,y
744,P7 - Paris,AAR-STR-CHA-314-2,AAR-STR-CHA-314,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_86,SACS,13,...,p.Asp202Gly,Het,P7_SACS_Paris_87,no,yes,no,NaN,13-23930146-T-C,n,y
746,P7 - Paris,AAR-SAL-GOL-61-19,NaN,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_88,SACS,13,...,p.Trp144ValfsTer39,Het,P7_SACS_Paris_89,no,yes,no,NaN,13-23939332-AAA-A,y,n
748,P7 - Paris,AAD-LYO-BUF-28-4,NaN,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_90,SACS,13,...,p.Leu96ThrfsTer3,Het,P7_SACS_Paris_91,no,yes,no,NaN,13-23942600-G-GT,n,y


In [103]:
df_solved[df_solved["case_status"]=="solved SACS"].drop_duplicates(subset=["submitter_id", "local_case_id"])

,submitter_id,local_case_id,local_family_id,prospax_case_id,ngs_database_id,main_phenotype,case_status,id,gene,chrom,...,cdna,prot_change,genotype,compound_het_id_s,paxgene_availability,pbmc_availability,fibroblasts_availability,comments,Variant_based_id,tier1_is_solved
0,P1 - Tübingen,ID16341,FN16341,064-258-853,TreatHSP,spastic ataxia,solved SACS,P1_SACS_Tubingen_1,SACS,13,...,c.3769G>T,p.Gly1275Ter,Het,P1_SACS_Tubingen_2,no,no,no,NaN,13-23914246-C-A,y
4,P1 - Tübingen,ID27288,FN27286,493-515-735,TreatHSP,spastic ataxia,solved SACS,P1_SACS_Tubingen_5,SACS,13,...,c.7162_7163del,p.Thr2388Argfs*10,Hom,NaN,no,no,no,NaN,13-23910851-GGT-G,y
27,P1 - Tübingen,ATX553,ATX553,NaN,Genesis,ataxia,solved SACS,P1_SACS_Tubingen_28,SACS,13,...,c.11707C>T,p.Arg3903Ter,Hom,NaN,unknown,unknown,unknown,NaN,13-23906308-G-A,y
28,P1 - Tübingen,AAR-SAL-BOU-196,AAR-SAL-BOU-196,NaN,Genesis,ataxia,solved SACS,P1_SACS_Tubingen_29,SACS,13,...,c.2018dup,p.Asn673LysfsTer33,Hom,NaN,unknown,unknown,unknown,NaN,13-23928732-A-AT,y
39,P1 - Tübingen,AAR-MAR-REB-111,AAR-MAR-REB-111,NaN,Genesis,ataxia,solved SACS,P1_SACS_Tubingen_40,SACS,13,...,c.12220G>C,p.Ala4074Pro,Hom,NaN,unknown,unknown,unknown,NaN,13-23905795-C-G,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,P7 - Paris,AAR-DIJ-REC-138-4,NaN,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_68,SACS,13,...,c.12973C>T,p.Arg4325Ter,Het,P7_SACS_Paris_69,no,yes,no,NaN,13-23905042-G-A,y
730,P7 - Paris,FSP-SAL-MAR-1562-17,NaN,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_70,SACS,13,...,c.11914C>T,p.Arg3972Ter,Het,P7_SACS_Paris_71,no,yes,no,NaN,13-23906101-G-A,y
736,P7 - Paris,AAR-SAL-DUC-225-8,NaN,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_76,SACS,13,...,c.4933C>T,p.Arg1645Ter,Het,P7_SACS_Paris_77,no,yes,no,NaN,13-23913082-G-A,y
746,P7 - Paris,AAR-SAL-GOL-61-19,NaN,NaN,Paris diagnostic pipeline,spastic ataxia,solved SACS,P7_SACS_Paris_88,SACS,13,...,c.429_430del,p.Trp144ValfsTer39,Het,P7_SACS_Paris_89,no,yes,no,NaN,13-23939332-AAA-A,y


In [104]:
df_sacs.drop_duplicates(subset=["submitter_id", "local_case_id"]).shape

(363, 25)

In [74]:
def return_all_words(alphabet, k, base=""):
    n = len(alphabet)
    recursive_function(alphabet, n, k, base)

def recursive_function(alphabet, length_alphabet, k, base):
    if k==0:
        print(base)
        return 
    for i in range(length_alphabet):
        new_word = base + alphabet[i]
        # print(new_word)
        recursive_function(alphabet, length_alphabet, k-1, new_word)

In [75]:
alpha = ["a", "b", "c"]
l = 2
for i in range(len(alpha)):
    return_all_words(alpha, i + 1)

a
b
c
aa
ab
ac
ba
bb
bc
ca
cb
cc
aaa
aab
aac
aba
abb
abc
aca
acb
acc
baa
bab
bac
bba
bbb
bbc
bca
bcb
bcc
caa
cab
cac
cba
cbb
cbc
cca
ccb
ccc


In [43]:
def permutations(l, n=-1, com_list=[]):
    # basis steps
    if n < 0:
        n = len(l)
    # base case of the recursive - when we reach n==0
    # return final state
    if n == 0:
        # at every call of function we decrement n 
        # if n = 0 it stops calling recursively the function
        return com_list
    
    # Recursive block
    # Decompose the original problem into simpler instances of the same problem
    sub_list=[]
    # very first recursive call
    # populate the list with all length=1 word
    if len(com_list)==0:
        for item in l:
            sub_list.append(item)
    else:
        # iterate on the set of letters
        for item in l:
            # uses the elements generated by the last recursive call
            for item2 in com_list:
                # concatenate the all words from last recursive call --> to the current letter from set
                sub_list.append(f"{item}{item2}")
    # at each return of the function the sublist is added to com_list
    # each call of the function generate words of ONE given length 
    return com_list + permutations(l, n-1, sub_list)

In [45]:
permutations(alpha)

['a',
 'b',
 'c',
 'aa',
 'ab',
 'ac',
 'ba',
 'bb',
 'bc',
 'ca',
 'cb',
 'cc',
 'aaa',
 'aab',
 'aac',
 'aba',
 'abb',
 'abc',
 'aca',
 'acb',
 'acc',
 'baa',
 'bab',
 'bac',
 'bba',
 'bbb',
 'bbc',
 'bca',
 'bcb',
 'bcc',
 'caa',
 'cab',
 'cac',
 'cba',
 'cbb',
 'cbc',
 'cca',
 'ccb',
 'ccc']

In [ ]:
# Python 3 program to print all
# possible strings of length k
	
# The method that prints all
# possible strings of length k.
# It is mainly a wrapper over
# recursive function printAllKLengthRec()
def printAllKLength(set, k):

	n = len(set)
	printAllKLengthRec(set, "", n, k)

# The main recursive method
# to print all possible
# strings of length k
def printAllKLengthRec(set, prefix, n, k):
	
	# Base case: k is 0,
	# print prefix
	if (k == 0) :
		print(prefix)
		return

	# One by one add all characters
	# from set and recursively
	# call for k equals to k-1
	for i in range(n):

		# Next character of input added
		newPrefix = prefix + set[i]
		
		# k is decreased, because
		# we have added a new character
		printAllKLengthRec(set, newPrefix, n, k - 1)

# Driver Code
if __name__ == "__main__":
	
	print("First Test")
	set1 = ['a', 'b']
	k = 3
	printAllKLength(set1, k)
	
	print("\nSecond Test")
	set2 = ['a', 'b', 'c', 'd']
	k = 1
	printAllKLength(set2, k)

# This code is contributed
# by ChitraNayal
